In [1]:
%matplotlib widget
import numpy as np
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import random

import matplotlib.pyplot as plt

import re

from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

from joblib import Parallel, delayed
import multiprocessing
from datetime import datetime

In [2]:
torch.set_printoptions(precision=10)

In [3]:
plikembd = open("../embeddings_small.txt")

embeddingscuda_chars = {}
embeddingscuda_chars['🈳'] = 0

#embeddingscuda = torch.tensor([]).to("cuda:0")
#embeddingscuda =  torch.cat((embeddingscuda, torch.zeros((400)).to("cuda:0")), 0)

#print(embeddingscuda.shape)

for cnt, line in enumerate(plikembd.readlines()):
    exp1 = line.replace("\n", "").split(":")
    
    znak = exp1[0]
    #embedding = torch.tensor([float(liczba) for liczba in exp1[1].replace(" ", "").replace("[", "").replace("]", "").split(",\t")[:-1]]).to("cuda:0")
    
    embeddingscuda_chars[znak] = cnt+1
   # embeddingscuda = torch.cat((embeddingscuda, embedding), 0)

#embeddingscuda = embeddingscuda.reshape(82, -1)

#print(embeddingscuda.shape)
#print(embeddingscuda[embeddingscuda_chars['a']])

In [4]:

polskie = open("../100k.txt", encoding='utf8')
slowa = [slowo.replace("\n", "") for slowo in polskie.readlines()]
polskie.close()
print(slowa[:100])
print(len(slowa))
slowa = slowa[:13000]

['abmicro', '527579', 'agro2011', 'nakamichi', 'dugant', '29082009', 'hannah1', 'ma12po', '580813', 'lukasz4', 'kriket', 'werohaze', 'dron14121987', 'pinot1', 'maria59', 'moltrex', 'kilim', 'Zabeczka', '7937', '7maluszek', 'gacek16', 'ruleta1', '17Cyprian', 'maj2006', 'o2skarek', 'anno01', 'gusciol', 'gumis1', 'malaz11', 'watomzon', 'rumianek', 'neoisp', 'zsofie', 'zipera', '141084', 'ultraboy', '110567', 'anbog', 'troyd34', 'roxboro', 'werunia', 'misiokrzysio', 'hydra6', 'grzes10', 'kakigewy', 'filip98', '20021970', 'natka777', 'patricia', 'polisnova', 'miko20', 'prj2501', 'grobian1', '1972ml', 'robo21101967', 'jw17061958', 'gregordj', 'insko2000', 'maclear', 'zl060374', 'asia7', 'misiu777', 'varius2', 'lgflatron', 'rele1', '130178', 'centrala', 'filoman', '41karat2', '84jaysinner', 'agatina', 'ambasada', 'maly17', 'goszkam', '1branka', 'dasti', 'petro111', '070165', '20111965', 'triforce', 'bogumil1', 'anna3005', 'dron10011001', 'santaclara', 'pas2526', '11011975', 'rest12', 'j87neo'

In [5]:
longestword = 0

unikalneznaki = []
for slowo in slowa:
    for literka in slowo:
        if literka not in unikalneznaki:
            unikalneznaki.append(literka)
    if len(slowo) > longestword:
        longestword = len(slowo)
            
print(unikalneznaki)
print(longestword)
print(len(unikalneznaki))

['a', 'b', 'm', 'i', 'c', 'r', 'o', '5', '2', '7', '9', 'g', '0', '1', 'n', 'k', 'h', 'd', 'u', 't', '8', 'p', '3', 'l', 's', 'z', '4', 'e', 'w', 'x', 'Z', '6', 'C', 'y', 'j', 'f', 'v', 'F', 'P', 'R', 'O', 'T', 'A', 'X', 'L', 'E', 'S', 'I', 'K', 'N', 'D', 'M', 'q', 'H', 'U', 'J', 'Q', 'G', 'W', 'B', 'V', 'Y', '@', '.', '#', '!', ';', '*', '`']
15
69


In [6]:
print(list(embeddingscuda_chars.keys()))
print(len(list(embeddingscuda_chars.keys())))

['🈳', ' ', '!', '#', '$', '%', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '=', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
82


In [7]:
onehotvectors = {}
onehotvectorsindexed = torch.tensor([]).to("cuda:0")
klasyid = {}

for cnt, litera in enumerate(list(embeddingscuda_chars.keys())):
    vector = [0 for __ in range(len(embeddingscuda_chars.keys()))]
    vector[cnt] = 1
    onehotvectors[litera] = torch.Tensor(vector).to("cuda:0")
    # onehotvectorsindexed.append(onehotvectors[litera])
    onehotvectorsindexed = torch.cat((onehotvectorsindexed, onehotvectors[litera]), 0)
    
    klasyid[litera] = cnt
    
onehotvectorsindexed = onehotvectorsindexed.reshape(len(onehotvectors), -1)
    
print(len(onehotvectors))
print(len(onehotvectors['🈳']))
print(len(klasyid))
print(klasyid)
print(onehotvectorsindexed[0])

82
82
82
{'🈳': 0, ' ': 1, '!': 2, '#': 3, '$': 4, '%': 5, '*': 6, '+': 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ';': 22, '=': 23, '@': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 37, 'N': 38, 'O': 39, 'P': 40, 'Q': 41, 'R': 42, 'S': 43, 'T': 44, 'U': 45, 'V': 46, 'W': 47, 'X': 48, 'Y': 49, 'Z': 50, '[': 51, ']': 52, '^': 53, '_': 54, '`': 55, 'a': 56, 'b': 57, 'c': 58, 'd': 59, 'e': 60, 'f': 61, 'g': 62, 'h': 63, 'i': 64, 'j': 65, 'k': 66, 'l': 67, 'm': 68, 'n': 69, 'o': 70, 'p': 71, 'q': 72, 'r': 73, 's': 74, 't': 75, 'u': 76, 'v': 77, 'w': 78, 'x': 79, 'y': 80, 'z': 81}
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.

In [8]:
print(onehotvectors['🈳'])
print(len(onehotvectors['🈳']))

dlugoschotvectora = len(onehotvectors['🈳'])

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
82


In [9]:
print("ilosc wejsc dyskryminatora: " + str(dlugoschotvectora * longestword))
#print("ilosc wyjsc generatora: " + str(len(embeddingscuda[0]) * longestword))

ilosc wejsc dyskryminatora: 1230


In [10]:
dataset_real = torch.Tensor([]).to("cuda:0")
iloscslow = 0

pusty = torch.zeros(len(embeddingscuda_chars.keys())).to("cuda:0")

for cnt in tqdm(range(len(slowa))):
    slowo = slowa[cnt]
    for index in range(longestword):
        if index<len(slowo):
            dataset_real = torch.cat((dataset_real, onehotvectors[slowo[index]]), 0)
        else:
            dataset_real = torch.cat((dataset_real, pusty), 0)
    iloscslow = iloscslow + 1

dataset_real = dataset_real.reshape(iloscslow, -1)

print(dataset_real.shape)

100%|████████████████████████████████████████████████████████████████████████████| 13000/13000 [04:13<00:00, 51.21it/s]


torch.Size([13000, 1230])


In [11]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()     
        
        self.fc_in = nn.Linear(1230, 1000)
        self.fc_h1 = nn.Linear(1000, 600)
        self.fc_h2 = nn.Linear(600, 100)
        self.fc_h3 = nn.Linear(100, 20)
#         self.fc_h4 = nn.Linear(100, 50)
        #self.fc_h5 = nn.Linear(200, 20)
        self.fc_out = nn.Linear(20, 1)
        
    def forward(self, x):      
        x = self.fc_in(x)
        x = F.leaky_relu(x)
        
        x = self.fc_h1(x)
        x = F.leaky_relu(x)
        
        x = self.fc_h2(x)
        x = F.leaky_relu(x)
        
        x = self.fc_h3(x)
        x = F.leaky_relu(x)
        
#         x = self.fc_h4(x)
#         x = F.relu(x)
        
#         x = self.fc_h5(x)
#         x = F.leaky_relu(x)
        
        x = self.fc_out(x)
        x = F.sigmoid(x)
        
        return x

In [12]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.fc_in = nn.Linear(10, 1000)
        self.bn1 = nn.BatchNorm1d(1000)
        self.fc_h1 = nn.Linear(1000, 1000)
        self.bn2 = nn.BatchNorm1d(1000)
        self.fc_h2 = nn.Linear(1000, 1000)
        self.bn3 = nn.BatchNorm1d(1000)
        self.fc_h3 = nn.Linear(1000, 1000)
        self.bn4 = nn.BatchNorm1d(1000)
        self.fc_h4 = nn.Linear(1000, 1000)
        self.bn5 = nn.BatchNorm1d(1000)
        self.fc_out = nn.Linear(1000, 1230)
        
        self.activation1 = nn.PReLU()
        self.softmax = nn.Softmax(dim=1)
        self.drop = nn.Dropout(p=0.4)
        
        self.p1 = torch.tensor([x for x in range(82)]).to("cuda:0")
        self.p2 = torch.tensor([x for x in range(82, 164)]).to("cuda:0")
        self.p3 = torch.tensor([x for x in range(164, 246)]).to("cuda:0")
        self.p4 = torch.tensor([x for x in range(246, 328)]).to("cuda:0")
        self.p5 = torch.tensor([x for x in range(328, 410)]).to("cuda:0")
        self.p6 = torch.tensor([x for x in range(410, 492)]).to("cuda:0")
        self.p7 = torch.tensor([x for x in range(492, 574)]).to("cuda:0")
        self.p8 = torch.tensor([x for x in range(574, 656)]).to("cuda:0")
        self.p9 = torch.tensor([x for x in range(656, 738)]).to("cuda:0")
        self.p10 = torch.tensor([x for x in range(738, 820)]).to("cuda:0")
        self.p11 = torch.tensor([x for x in range(820, 902)]).to("cuda:0")
        self.p12 = torch.tensor([x for x in range(902, 984)]).to("cuda:0")
        self.p13 = torch.tensor([x for x in range(984, 1066)]).to("cuda:0")
        self.p14 = torch.tensor([x for x in range(1066, 1148)]).to("cuda:0")
        self.p15 = torch.tensor([x for x in range(1148, 1230)]).to("cuda:0")
        
        #         x1 = torch.argmax(self.softmax(x[:,0:82]), dim=1)
#         x2 = torch.argmax(self.softmax(x[:,82:164]), dim=1)
#         x3 = torch.argmax(self.softmax(x[:,164:246]), dim=1)
#         x4 = torch.argmax(self.softmax(x[:,246:328]), dim=1)
#         x5 = torch.argmax(self.softmax(x[:,328:410]), dim=1)
#         x6 = torch.argmax(self.softmax(x[:,410:492]), dim=1)
#         x7 = torch.argmax(self.softmax(x[:,492:574]), dim=1)
#         x8 = torch.argmax(self.softmax(x[:,574:656]), dim=1)
#         x9 = torch.argmax(self.softmax(x[:,656:738]), dim=1)
#         x10 = torch.argmax(self.softmax(x[:,738:820]), dim=1)
#         x11 = torch.argmax(self.softmax(x[:,820:902]), dim=1)
#         x12 = torch.argmax(self.softmax(x[:,902:984]), dim=1)
#         x13 = torch.argmax(self.softmax(x[:,984:1066]), dim=1)
#         x14 = torch.argmax(self.softmax(x[:,1066:1148]), dim=1)
#         x15 = torch.argmax(self.softmax(x[:,1148:1230]), dim=1)
        
    def forward(self, x):
        
        x = self.fc_in (x)
        x1 = F.relu(x)
        
        x1 = self.bn1(x1)
        
        x = self.fc_h1(x1)
        x2 = F.relu(x)
        
        x2 = self.bn2(x2)
        
        x = x1 * x2
        
        #x = self.drop(x)
          
        x = self.fc_h2(x)
        x3 = F.relu(x)
        
        x3 = self.bn1(x3)
        
        x = x1 * x2 * x3
        
        #x = self.drop(x)
        
        x = self.fc_h3(x)
        x4 = F.relu(x)
        
        x4 = self.bn1(x4)
        
        x = x1 * x2 * x3 * x4
        
        #x = self.drop(x)
        
        x = self.fc_h4(x)
        x5 = F.relu(x)
        
        x5 = self.bn1(x5)
        
        x = x1 * x2 + x3 * x4 * x5
        
        x = self.fc_out(x)
        x = F.relu(x)

        
        x1 = torch.argmax(self.softmax(torch.index_select(x, 1, self.p1)), dim=1)
        x2 = torch.argmax(self.softmax(torch.index_select(x, 1, self.p2)), dim=1)
        x3 = torch.argmax(self.softmax(torch.index_select(x, 1, self.p3)), dim=1)
        x4 = torch.argmax(self.softmax(torch.index_select(x, 1, self.p4)), dim=1)
        x5 = torch.argmax(self.softmax(torch.index_select(x, 1, self.p5)), dim=1)
        x6 = torch.argmax(self.softmax(torch.index_select(x, 1, self.p6)), dim=1)
        x7 = torch.argmax(self.softmax(torch.index_select(x, 1, self.p7)), dim=1)
        x8 = torch.argmax(self.softmax(torch.index_select(x, 1, self.p8)), dim=1)
        x9 = torch.argmax(self.softmax(torch.index_select(x, 1, self.p9)), dim=1)
        x10 = torch.argmax(self.softmax(torch.index_select(x, 1, self.p10)), dim=1)
        x11 = torch.argmax(self.softmax(torch.index_select(x, 1, self.p11)), dim=1)
        x12 = torch.argmax(self.softmax(torch.index_select(x, 1, self.p12)), dim=1)
        x13 = torch.argmax(self.softmax(torch.index_select(x, 1, self.p13)), dim=1)
        x14 = torch.argmax(self.softmax(torch.index_select(x, 1, self.p14)), dim=1)
        x15 = torch.argmax(self.softmax(torch.index_select(x, 1, self.p15)), dim=1)
        
        
        ##todo przyspieszyć
        
        
        #tt = torch.tensor([[-1], [0, 1,2,3,4,5,6]])
        
#         x1 = torch.argmax(self.softmax(x[:,0:82]), dim=1)
#         x2 = torch.argmax(self.softmax(x[:,82:164]), dim=1)
#         x3 = torch.argmax(self.softmax(x[:,164:246]), dim=1)
#         x4 = torch.argmax(self.softmax(x[:,246:328]), dim=1)
#         x5 = torch.argmax(self.softmax(x[:,328:410]), dim=1)
#         x6 = torch.argmax(self.softmax(x[:,410:492]), dim=1)
#         x7 = torch.argmax(self.softmax(x[:,492:574]), dim=1)
#         x8 = torch.argmax(self.softmax(x[:,574:656]), dim=1)
#         x9 = torch.argmax(self.softmax(x[:,656:738]), dim=1)
#         x10 = torch.argmax(self.softmax(x[:,738:820]), dim=1)
#         x11 = torch.argmax(self.softmax(x[:,820:902]), dim=1)
#         x12 = torch.argmax(self.softmax(x[:,902:984]), dim=1)
#         x13 = torch.argmax(self.softmax(x[:,984:1066]), dim=1)
#         x14 = torch.argmax(self.softmax(x[:,1066:1148]), dim=1)
#         x15 = torch.argmax(self.softmax(x[:,1148:1230]), dim=1)
        
        
        
#         x3 = torch.argmax(self.softmax(x[:][164:246]))
#         x4 = torch.argmax(self.softmax(x[:][246:328]))
#         x5 = torch.argmax(self.softmax(x[:][328:410]))
#         x6 = torch.argmax(self.softmax(x[:][410:492]))
#         x7 = torch.argmax(self.softmax(x[:][492:574]))
#         x8 = torch.argmax(self.softmax(x[:][574:656]))
#         x9 = torch.argmax(self.softmax(x[:][656:738]))
#         x10 = torch.argmax(self.softmax(x[:][738:820]))
#         x11 = torch.argmax(self.softmax(x[:][820:902]))
#         x12 = torch.argmax(self.softmax(x[:][902:984]))
#         x13 = torch.argmax(self.softmax(x[:][984:1066]))
#         x14 = torch.argmax(self.softmax(x[:][1066:1148]))
#         x15 = torch.argmax(self.softmax(x[:][1148:1230]))
        
        
        return torch.cat((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15), dim=0).reshape(len(x), -1)
        #return torch.tensor([x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15])
        #return torch.argmax(self.softmax(x[:,0:82]), dim=1)
        
        #return x

In [13]:
discriminator = Discriminator().to("cuda:0")
generator = Generator().to("cuda:0")

TensorReal = dataset_real

TensorFakeOuts = torch.zeros(len(dataset_real)).to("cuda:0")
TensorRealOuts = torch.ones(len(dataset_real)).to("cuda:0")

TensorOuts = torch.cat((TensorFakeOuts, TensorRealOuts), 0).to("cuda:0")

In [14]:
criterion = nn.BCELoss()
criterion_segment = nn.CrossEntropyLoss()

In [15]:
TensorFakeSeeds = torch.rand(iloscslow, 10).to("cuda:0")
TensorFakeIns = generator(TensorFakeSeeds[0:2]).to("cuda:0")

In [16]:
print(TensorFakeIns.shape)
print(TensorFakeIns)
print(TensorFakeIns[0])

torch.Size([2, 15])
tensor([[49, 49, 59, 59, 57, 57, 11, 11,  8,  8,  6,  6, 57, 57, 61],
        [61,  5,  5, 13, 13, 55, 55, 47, 47, 33, 33, 43, 43, 56, 56]],
       device='cuda:0')
tensor([49, 49, 59, 59, 57, 57, 11, 11,  8,  8,  6,  6, 57, 57, 61],
       device='cuda:0')


In [17]:
print(onehotvectorsindexed[TensorFakeIns[0][0].item()])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')


In [18]:
indices = torch.tensor([x for x in range(82)]).to("cuda:0")
torch.index_select(TensorFakeIns, 1, indices).shape

torch.Size([2, 82])

In [19]:
def outputtoonehots(TensorFakes, printletters=False):
#     zwrot = torch.tensor([]).to("cuda:0")
#     for cnt in range(len(TensorFakes)):
#         zwrot = torch.cat(
#             (
#                 zwrot,
#                 onehotvectorsindexed[fake[0].item()],
#                 onehotvectorsindexed[fake[1].item()],
#                 onehotvectorsindexed[fake[2].item()],
#                 onehotvectorsindexed[fake[3].item()],
#                 onehotvectorsindexed[fake[4].item()],
#                 onehotvectorsindexed[fake[5].item()],
#                 onehotvectorsindexed[fake[6].item()],
#                 onehotvectorsindexed[fake[7].item()],
#                 onehotvectorsindexed[fake[8].item()],
#                 onehotvectorsindexed[fake[9].item()],
#                 onehotvectorsindexed[fake[10].item()],
#                 onehotvectorsindexed[fake[11].item()],
#                 onehotvectorsindexed[fake[12].item()],
#                 onehotvectorsindexed[fake[13].item()],
#                 onehotvectorsindexed[fake[14].item()],
#             )
#         , dim=0)
#         zwrot = torch.cat(
#             (
#                 zwrot,
#                 torch.index_select(onehotvectorsindexed, 0, TensorFakes[cnt])
#             ), dim=0)

    if printletters:
        fake = TensorFakes[0]
        print(list(embeddingscuda_chars.keys())[fake[0].item()],
             list(embeddingscuda_chars.keys())[fake[1].item()],
             list(embeddingscuda_chars.keys())[fake[2].item()],
             list(embeddingscuda_chars.keys())[fake[3].item()],
             list(embeddingscuda_chars.keys())[fake[4].item()],
             list(embeddingscuda_chars.keys())[fake[5].item()],
             list(embeddingscuda_chars.keys())[fake[6].item()],
             list(embeddingscuda_chars.keys())[fake[7].item()],
             list(embeddingscuda_chars.keys())[fake[8].item()],
             list(embeddingscuda_chars.keys())[fake[9].item()],
             list(embeddingscuda_chars.keys())[fake[10].item()],
             list(embeddingscuda_chars.keys())[fake[11].item()],
             list(embeddingscuda_chars.keys())[fake[12].item()],
             list(embeddingscuda_chars.keys())[fake[13].item()],
             list(embeddingscuda_chars.keys())[fake[14].item()])

    zwrot = torch.index_select(onehotvectorsindexed, 0, TensorFakes.reshape(-1))
    
    return zwrot.reshape(len(TensorFakes), -1)

In [20]:
xx = outputtoonehots(TensorFakeIns[:2]); xx.shape

torch.Size([2, 1230])

In [36]:
torch.mean(TensorFakeIns[:50].gt(1).float().reshape(-1), 0)*10

tensor(10., device='cuda:0')

In [37]:
def myloss(Fakes):
    return torch.mean(Fakes.gt(1).float().reshape(-1), 0)*10

In [38]:
#discriminator(xx)

In [39]:
optimizerGenerator = optim.Adam(generator.parameters(), lr=3e-3)
optimizerClassifier = optim.Adam(discriminator.parameters(), lr=3e-3)

In [41]:
for epoch in range(20000):
    TensorFakeSeeds = torch.rand(iloscslow, 10).to("cuda:0")
    TensorFakeIns = generator(TensorFakeSeeds).to("cuda:0")
    
    printed = False
    if epoch%2==0:
        printed = True
    
    Fakes = outputtoonehots(TensorFakeIns, printletters=printed)
    
    TensorIns = torch.cat((Fakes, TensorReal), 0).to("cuda:0")
    
    ## DISCRIMINATOR
    discriminator.train()
    optimizerClassifier.zero_grad()
    
    y1_ = discriminator(TensorIns)
    lossDiscriminator = criterion(y1_, TensorOuts)
    
    lossDiscriminator.backward()
    optimizerClassifier.step()
    
    
    ## GENERATOR
    generator.train()
    optimizerGenerator.zero_grad()
    
    y2_ = discriminator(Fakes)
    lossGenerator = (criterion(y2_, TensorRealOuts) + myloss(TensorFakeIns))
    
    lossGenerator.backward()
    optimizerGenerator.step()
    
    if epoch%2==0:
        print("Loss discriminator:" + str(lossDiscriminator.item()), "Loss generator:" + str(lossGenerator.item()))

RuntimeError: CUDA out of memory. Tried to allocate 49.63 MiB (GPU 0; 3.00 GiB total capacity; 1.64 GiB already allocated; 14.10 MiB free; 113.39 MiB cached)